# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 45 new papers today
          18 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/17 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2301.08258


extracting tarball to tmp_2301.08258...

 done.
Retrieving document from  https://arxiv.org/e-print/2301.08260


extracting tarball to tmp_2301.08260...

...

 done.
Retrieving document from  https://arxiv.org/e-print/2301.08673


extracting tarball to tmp_2301.08673...

 done.
Retrieving document from  https://arxiv.org/e-print/2301.08692


extracting tarball to tmp_2301.08692...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.08310-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.08310) | **Gaia-ESO Survey: massive stars in the Carina Nebula. A new census of OB  stars**  |
|| S. R. Berlanas, et al. -- incl., <mark>A. Herrero</mark>, <mark>L. Mahy</mark>, <mark>M. Bergemann</mark> |
|*Appeared on*| *2023-01-23*|
|*Comments*| *Accepted for publication in A&A, 62 pages, 8 figures*|
|**Abstract**| The Gaia-ESO survey sample of massive OB stars in the Carina Nebula consists of 234 stars. The addition of brighter sources from the Galactic O-Star Spectroscopic Survey and additional sources from the literature allows us to create the most complete census of massive OB stars done so far in the region. It contains a total of 316 stars, being 18 of them in the background and four in the foreground. Of the 294 stellar systems in Car OB1, 74 are of O type, 214 are of non-supergiant B type and 6 are of WR or non-O supergiant (II to Ia) spectral class. We identify 20 spectroscopic binary systems with an O-star primary, of which 6 are reported for the first time, and another 18 with a B-star primary, of which 13 are new detections. The average observed double-lined binary fraction of O-type stars in the surveyed region is 0.35, which represents a lower limit. We find a good correlation between the spectroscopic n-qualifier and the projected rotational velocity of the stars. The fraction of candidate runaways among the stars with and without the n-qualifier is 4.4% and 2.4%, respectively, although non resolved double-lined binaries can be contaminating the fast rotators sample. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.08258-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.08258) | **Towards Robust Atmospheric Retrieval on Cloudy L Dwarfs: The Impact of  Thermal and Abundance Profile Assumptions**  |
|| Melanie Rowland, Caroline Morley, <mark>Michael Line</mark> |
|*Appeared on*| *2023-01-23*|
|*Comments*| *20 pages, 12 figures, 5 tables, submitted to ApJ*|
|**Abstract**| Constraining L dwarf properties from their spectra is challenging. Near-infrared spectra probe a limited range of pressures, while many species condense within their photospheres. Condensation creates two complexities: gas-phase species "rain out" (decreasing in abundances by many orders of magnitude) and clouds form. We designed tests using synthetic data to determine the best approach for retrieving L dwarf spectra, isolating the challenges in the absence of cloud opacity. We conducted atmospheric retrievals on synthetic cloud-free L dwarf spectra derived from the Sonora Bobcat models at SpeX resolution using a variety of thermal and chemical abundance profile parameterizations. For objects hotter than L5 (T$_{eff}$ ~ 1700 K), the limited pressure layers probed in the near-IR are mostly convective; parameterized PT profiles bias results and free, unsmoothed profiles should be used. Only when many layers both above and below the radiative-convective boundary are probed can parameterized profiles provide accurate results. Furthermore, a nonuniform abundance profile for iron hydride (FeH) is needed to accurately retrieve bulk properties of early- to mid- L dwarfs. Nonuniform prescriptions for other gases in near-IR retrievals may also be warranted near the L/T transition (CH$_{4}$) and early Y dwarfs (Na and K). We demonstrate the utility of using realistic self-consistent models to benchmark retrievals and suggest how they can be used in the future. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.08260-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.08260) | **$S_8$ Tension in the Context of Dark Matter-Baryon Scattering**  |
|| <mark>Adam He</mark>, Mikhail M. Ivanov, Rui An, Vera Gluscevic |
|*Appeared on*| *2023-01-23*|
|*Comments*| *21 pages, 10 figures, submitted to PRL*|
|**Abstract**| We explore the concordance of cosmological data in the context of dark matter (DM) that interacts with baryons. Using the effective theory of large-scale structure, we perform the first analysis of galaxy clustering data for this scenario and find a mild $\sim 3\sigma$ preference for velocity-independent DM-baryon scattering, assuming 10% of DM is interacting. Our results indicate that a broad power suppression on small scales is a generic feature that may help resolve $S_8$ tension between cosmological data sets. The validity of this interacting DM model will be critically tested with incoming survey data. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.08512-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.08512) | **A Unified Model for Bipolar Outflows from Young Stars: Apparent Magnetic  Jet Acceleration**  |
|| Hsien Shang, Ruben Krasnopolsky, <mark>Chun-Fan Liu</mark> |
|*Appeared on*| *2023-01-23*|
|*Comments*| *21 pages, 8 figures, 1 table, to appear in Astrophysical Journal Letters (2023)*|
|**Abstract**| We explore a new, efficient mechanism that can power toroidally magnetized jets up to two to three times their original terminal velocity after they enter a self-similar phase of magnetic acceleration. Underneath the elongated outflow lobe formed by a magnetized bubble, a wide-angle free wind, through the interplay with its ambient toroid, is compressed and accelerated around its axial jet. The extremely magnetic bubble can inflate over its original size, depending on the initial Alfv\'en Mach number $M_A$ of the launched flow. The shape-independent slope $\partial{}v_r/\partial{}r=2/3t$ is a salient feature of the self-similarity in the acceleration phase. Peculiar kinematic signatures are observable in the position--velocity (PV) diagrams and can combine with other morphological signatures as probes for the density-collimated jets arising in toroidally dominated magnetized winds. The apparent second acceleration is powered by the decrease of the toroidal magnetic field but operates far beyond the scales of the primary magnetocentrifugal launch region and the free asymptotic terminal state. Rich implications may connect the jets arising from the youngest protostellar outflows such as HH 211 and HH 212 and similar systems with parsec-scale jets across the mass and evolutionary spectra. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.08673-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.08673) | **Disintegration of Long-Period Comet C/2021 A1 (Leonard)**  |
|| David Jewitt, et al. -- incl., <mark>Jing Li</mark> |
|*Appeared on*| *2023-01-23*|
|*Comments*| *35 pages, 7 figures*|
|**Abstract**| We present imaging observations of the disintegrating long-period comet C/2021 A1 (Leonard). High resolution observations with Hubble Space Telescope show no evidence for surviving fragments, and place a 3 sigma upper limit to their possible radius about 60 m (albedo 0.1 assumed). In contrast, wide field observations from the Swan Hill Observatory, Australia, show an extensive debris cloud, the cross-section and estimated mass of which are consistent with complete disintegration of the nucleus near mid- December 2021 (at about 0.8 au). Two methods give the pre-disruption nucleus radius, r = 0.6+/-0.2 km. Tidal, collisional, sublimation and pressure-confined explosion models provide implausible explanations of the disintegration. However, rotational instability driven by outgassing torques has a very short timescale (of order 0.1 year) given the orbit and size of the C/2021 A1 nucleus, and offers the most plausible mechanism for the disruption. Initial rotational breakup is accelerated by the exposure and strong sublimation of previously buried volatiles, leading to catastrophic destruction of the nucleus. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.08692-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.08692) | **Constraints on $S_8$ from a full-scale and full-shape analysis of  redshift-space clustering and galaxy-galaxy lensing in BOSS**  |
|| <mark>Johannes U. Lange</mark>, et al. -- incl., <mark>Andrew P. Hearin</mark>, <mark>Frank C. van den Bosch</mark>, <mark>Hong Guo</mark>, <mark>Joseph DeRose</mark> |
|*Appeared on*| *2023-01-23*|
|*Comments*| *22 pages, 16 figures, submitted to MNRAS, comments welcome*|
|**Abstract**| We present a novel simulation-based cosmological analysis of galaxy-galaxy lensing and galaxy redshift-space clustering. Compared to analysis methods based on perturbation theory, our simulation-based approach allows us to probe a much wider range of scales, $0.4 \, h^{-1} \, \mathrm{Mpc}$ to $63 \, h^{-1} \, \mathrm{Mpc}$, including highly non-linear scales, and marginalises over astrophysical effects such as assembly bias. We apply this framework to data from the Baryon Oscillation Spectroscopic Survey LOWZ sample cross-correlated with state-of-the-art gravitational lensing catalogues from the Kilo Degree Survey and the Dark Energy Survey. We show that gravitational lensing and redshift-space clustering when analysed over a large range of scales place tight constraints on the growth-of-structure parameter $S_8 = \sigma_8 \sqrt{\Omega_{\rm m} / 0.3}$. Overall, we infer $S_8 = 0.792 \pm 0.022$ when analysing the combination of galaxy-galaxy lensing and projected galaxy clustering and $S_8 = 0.771 \pm 0.027$ for galaxy redshift-space clustering. These findings highlight the potential constraining power of full-scale studies over studies analysing only large scales, and also showcase the benefits of analysing multiple large-scale structure surveys jointly. Our inferred values for $S_8$ fall below the value inferred from the CMB, $S_8 = 0.834 \pm 0.016$. While this difference is not statistically significant by itself, our results mirror other findings in the literature whereby low-redshift large scale structure probes infer lower values for $S_8$ than the CMB, the so-called $S_8$-tension. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.08265-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.08265) | **On the distribution of the CNM in spiral galaxies**  |
|| <mark>Rowan J. Smith</mark>, et al. -- incl., <mark>Patrick Hennebelle</mark> |
|*Appeared on*| *2023-01-23*|
|*Comments*| *13 pages, 19 figures, submitted to MNRAS*|
|**Abstract**| The Cold Neutral Medium (CNM) is an important part of the galactic gas cycle and a precondition for the formation of molecular and star forming gas, yet its distribution is still not fully understood. In this work we present extremely high resolution simulations of spiral galaxies with time-dependent chemistry such that we can track the formation of the CNM, its distribution within the galaxy, and its correlation with star formation. We find no strong radial dependence between the CNM fraction and total HI due to the decreasing interstellar radiation field counterbalancing the decreasing gas column density at larger galactic radii.However, the CNM fraction does increase in spiral arms where the CNM distribution is clumpy, rather than continuous, overlapping more closely with H2. The CNM doesn't extend out radially as far as HI, and the vertical scale height is smaller in the outer galaxy compared to HI with no flaring. The CNM column density scales with total midplane pressure and disappears from the gas phase below values of PT/kB =1000 K/cm3. We find that the star formation rate density follows a similar scaling law with CNM column density to the total gas Kennicutt-Schmidt law. In the outer galaxy we produce realistic vertical velocity dispersions in the HI purely from galactic dynamics but our models do not predict CNM at the extremely large radii observed in HI absorption studies of the Milky Way. We suggest that extended spiral arms might produce isolated clumps of CNM at these radii. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Max' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.08640-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.08640) | **Tracing the Evolution of SMBHs and Stellar Objects in Galaxy Mergers: An  Multi-mass Direct N-body Model**  |
|| <mark>Shuo Li</mark>, et al. -- incl., <mark>F.K. Liu</mark> |
|*Appeared on*| *2023-01-23*|
|*Comments*| *36 pages, 11 figures, and 1 table. Accepted for publication in ApJ*|
|**Abstract**| By using direct N-body numerical simulations, we model the dynamical co-evolution of two supermassive black holes (SMBHs) and the surrounding stars in merging galaxies. In order to investigate how different stellar components evolve during the merger, we generate evolved stellar distributions with an initial mass function. Special schemes have also been developed to deal with some rare but interesting events, such as tidal disruption of main sequence stars, the plunge of low mass stars, white dwarfs, neutron stars and stellar mass black holes, and the partial tidal disruption of red giants or asymptotic giant branch stars. Our results indicate that the formation of a bound supermassive black hole binary (SMBHB) will enhance the capture rates of stellar objects by the SMBHs. Compared to the equal stellar mass model, the multi-mass model tends to result in a higher average mass of disrupted stars. Instead of being tidally disrupted by the SMBH, roughly half of the captured main sequence stars will directly plunge into the SMBH because of their small stellar radius. Giant stars, on the other hand, can be stripped of their envelopes if they are close enough to the SMBH. Though most remnants of the giant stars can survive after the disruption, a small fraction still could plunge into the SMBH quickly or after many orbital periods. Our results also indicate significant mass segregation of compact stars at the beginning of the merger, and then this effect is destroyed as the two SMBHs form a bound binary. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2301.08310.md
    + _build/html/tmp_2301.08310/./Gaia_CMD.png
    + _build/html/tmp_2301.08310/./pm_vsini_new4_1.png
    + _build/html/tmp_2301.08310/./pm_vsini_new4_2.png
    + _build/html/tmp_2301.08310/./location_new2.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\GG}{\mbox{G_3}}$
$\newcommand{\GGc}{\mbox{G^\prime_3}}$
$\newcommand{\BP}{\mbox{G_{\rm BP3}}}$
$\newcommand{\RP}{\mbox{G_{\rm RP3}}}$
$\newcommand{\BPRP}{\mbox{G_{\rm BP3}-G_{\rm RP3}}}$
$\newcommand{\pic}{\mbox{\varpi_{\rm c}}}$
$\newcommand{\mci}[1]{\multicolumn{1}{c}{#1}}$
$\newcommand{\VO}[1]{Villafranca~O-{#1}}$
$\newcommand{\EBV}{\mbox{E(4405-5495)}}$
$\newcommand{\RV}{\mbox{R_{5495}}}$
$\newcommand{\lili}{\mbox{LiLiMaRlin}}$
$\newcommand{\kms}{km s^{-1}}$
$\newcommand{\Teff}{\mbox{T_{\rm eff}}}$
$\newcommand{\bibname}{References}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\GG$}{\mbox{G_3}}$
$\newcommand{$\GG$c}{\mbox{G^\prime_3}}$
$\newcommand{$\BP$}{\mbox{G_{\rm BP3}}}$
$\newcommand{$\RP$}{\mbox{G_{\rm RP3}}}$
$\newcommand{$\BP$RP}{\mbox{G_{\rm BP3}-G_{\rm RP3}}}$
$\newcommand{$\pic$}{\mbox{\varpi_{\rm c}}}$
$\newcommand{$\mci$}[1]{\multicolumn{1}{c}{#1}}$
$\newcommand{$\VO$}[1]{Villafranca~O-{#1}}$
$\newcommand{$\EBV$}{\mbox{E(4405-5495)}}$
$\newcommand{$\RV$}{\mbox{R_{5495}}}$
$\newcommand{$\lili$}{\mbox{LiLiMaRlin}}$
$\newcommand{$\kms$}{km s^{-1}}$
$\newcommand{$\Teff$}{\mbox{T_{\rm eff}}}$
$\newcommand{$\bibname$}{References}$</div>



<div id="title">

# Gaia-ESO Survey: massive stars in the Carina Nebula : I. A new census of OB stars

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2301.08310-b31b1b.svg)](https://arxiv.org/abs/2301.08310)<mark>Appeared on: 2023-01-23</mark> - _Accepted for publication in A&A, 62 pages, 8 figures_

</div>
<div id="authors">

S. R. Berlanas, et al. -- incl., <mark><mark>A. Herrero</mark></mark>, <mark><mark>L. Mahy</mark></mark>, <mark><mark>M. Bergemann</mark></mark>

</div>
<div id="abstract">

**Abstract:** The Carina Nebula is one of the major massive star-forming regions in the Galaxy. Its relatively nearby distance (2.35 kpc) makes it an ideal laboratory for the study of massive star formation, structure and evolution, both for individual stars and stellar systems.   Thanks to the high-quality spectra provided by Gaia-ESO survey and the LiLiMaRlin library, as well as Gaia EDR3 astrometry, a detailed and homogeneous spectroscopic characterization of its massive stellar content can be carried out.Our main objective is to spectroscopically characterize all massive members of the Carina Nebula in the Gaia-ESO survey footprint to provide an updated census of massive stars in the region and an updated estimate of the binary fraction of O stars.We perform accurate spectral classification by using an interactive code that compares spectra with spectral libraries of OB standards, as well as line-based classic methods. Membership is calculated using our own algorithm based on Gaia EDR3 astrometry. To check the correlation between the spectroscopic n-qualifier and the rotational velocity, we use the semi-automated tool for the line-broadening characterization of OB stars  which is based on a combined Fourier Transform and Goodness-of-fit methodology.The Gaia-ESO survey sample of massive OB stars in the Carina Nebula consists of 234 stars. The addition of brighter sources from the Galactic O-Star Spectroscopic Survey and additional sources from the literature allows us to create the most complete census of massive OB stars done so far in the region. It contains a total of   316 stars, being 18 of them in the background and four in the foreground. Of the 294 stellar systems in Car OB1, 74 are of O type, 214 are of non-supergiant B type and 6 are of WR or non-O supergiant (II to Ia) spectral class. We identify 20 spectroscopic binary systems with an O-star primary, of which 6 are reported for the first time, and another 18 with a B-star primary, of which 13 are new detections. The average observed double-lined binary fraction of O-type stars in the surveyed region is 0.35, which represents a lower limit. We find a good correlation between the spectroscopic n-qualifier and the projected rotational velocity of the stars. The fraction of candidate runaways among the stars with and without the n-qualifier is 4.4$\%$and 2.4$\%$, respectively, although non resolved double-lined binaries can be contaminating the fast rotators sample.

</div>

<div id="div_fig1">

<img src="tmp_2301.08310/./Gaia_CMD.png" alt="Fig3" width="100%"/>

**Figure 3. -** First panel, see next page for the second one: {\it Gaia} EDR3 CMD for the stars with spectral types in this paper. Different symbols and colors are used to represent stars with parallaxes compatible with being (or otherwise assumed to be) in the foreground (4), in the background (18), or in Car OB1 (294). Of the Car OB1 stars, 6 are of Wolf-Rayet or non-O supergiant (II to Ia) spectral class, 74 are of O type, and 214 are of non-supergiant B type. Four of the \citet{Preietal21} stars are outside the frame towards the lower right due to their high extinction. Black lines show the average main sequence at a distance of 2.35 kpc with no extinction and with values of $\EBV$ of 0.5, 1.0, 1.5, and 2.0 (labelled) using the extinction law of \citet{Maizetal14a} with a value of $\RV$ of 4.5, which is typical of the region but with a large dispersion  \citep{MaizBarb18}. Solid orange lines show the $\RV$ = 4.5 extinction tracks for average MS stars of $\Teff$ of 52.5 kK, 30 kK, and 20 kK (labelled), respectively. The dotted orange line shows the $\RV$ = 3.0 extinction track for $\Teff$ = 30 kK. (*Gaia_CMD*)

</div>
<div id="div_fig2">

<img src="tmp_2301.08310/./pm_vsini_new4_1.png" alt="Fig6.1" width="50%"/><img src="tmp_2301.08310/./pm_vsini_new4_2.png" alt="Fig6.2" width="50%"/>

**Figure 6. -** Proper motion distribution from Gaia EDR3 astrometry for all stars of our census in each assigned Villafranca group. Orange squares indicate those OB stars analyzed in this work that are rotating at $v\sin i$$\geq$ 200 km s$^{-1}$. Blue crosses represent identified binary systems. Circles represent group proper motion constraints, whose centers $\mu_{\alpha *,g}$  and  $\mu_{\delta,g}$ are those shown in Table \ref{pm_groups} in the central columns. For comparison, red plus symbols indicate group centers from Villafranca II and III works. Note that stars labelled as Car OB1 members are not included in the panels. (*pm_fast*)

</div>
<div id="div_fig3">

<img src="tmp_2301.08310/./location_new2.png" alt="Fig2" width="100%"/>

**Figure 2. -** Negative image of the Great Carina Nebula by Robert Gendler and Stephane Guisard showing the location of the whole census of massive stars in the GES surveyed area presented in this work. Yellow and cyan colors indicate O and B-type stars, respectively. Green, red, purple and pink colors have been used to represent the sdO, LBV,  WR and RSG stars, respectively. Small filled-circles refer to the GES sample while rhombuses and squares refer to stars from GOSSS/LiLiMarlin and other works \citep[][]{smith06a,Alexetal16,Preietal21} not present in GES, respectively. Red circles indicate the observing GES pointings while the blue ones indicate the Villafranca groups: O-002 (Trumpler 14), O-003 (Trumpler 16 W), O-025 (Trumpler 16 E), O-027 (Trumpler 15), O-028 (Collinder 228), O-029 (Collinder 232), and O-030 (Bochum 11). The V-shaped extinction lane that dominates the appearance of the nebula is clearly seen crossing the image from top to bottom. (*carina*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

177  publications in the last 7 days.
	 _build/html/2301.08310.md
	 _build/html/2301.07990.md
	 _build/html/2301.07366.md
	 _build/html/2301.07199.md
	 _build/html/2301.07116.md
	 _build/html/2301.06873.md
	 _build/html/2301.06819.md
	 _build/html/2301.06575.md
	 _build/html/2301.05720.md
	 _build/html/2301.05718.md
	 _build/html/2301.05034.md
	 _build/html/2301.04656.md
	 _build/html/2301.04533.md
	 _build/html/2301.04442.md
	 _build/html/2301.03811.md
	 _build/html/2301.03689.md
	 _build/html/2301.03387.md
	 _build/html/2301.03209.md
	 _build/html/2301.02671.md
	 _build/html/2301.02477.md
	 _build/html/2301.02313.md
	 _build/html/2301.02265.md
	 _build/html/2301.01937.md
	 _build/html/2301.01895.md
	 _build/html/2301.01684.md
	 _build/html/2301.01306.md
	 _build/html/2301.00881.md
	 _build/html/2301.00854.md
	 _build/html/2301.00578.md
	 _build/html/2212.12039.md
	 _build/html/2212.11991.md
	 _build/html/2212.11425.md
	 _build/html/2212.11177.md
	 _build/html/2212.10804.md
	 _build/h

In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

10  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers